# Importing data and libraries

In [ ]:
import numpy as np
import pandas as pd

import re

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
df_auto = pd.read_excel('Pre Processing/automobile.xlsx', index_col=0)
df_fashion = pd.read_excel('Pre Processing/fashion.xlsx', index_col=0)
df_finance = pd.read_excel('Pre Processing/finance.xlsx', index_col=0)
df_tech = pd.read_excel('Pre Processing/tech.xlsx', index_col=0)

Remark: For smooth code execution, it is recommended to download all the excel files required in a folder named "Pre Processing" and the folder should be in the same directory as this Jupyter Notebook file. Otherwise, it is necessary to change the file paths accordingly.

# Merging the dataframes

In [ ]:
# We merge all the sectors into one dataframe
frames = [df_auto, df_fashion, df_finance, df_tech]
df = pd.concat(frames)

# Data pre-processing

### Dropping duplicate rows

In [ ]:
# We drop duplicate rows as they hold redundant information
print('Shape before : ', df.shape)
print('Number of duplicated rows : ', df[df.duplicated()].shape[0])
df = df.drop_duplicates()
print('Shape after : ', df.shape)

Shape before :  (129866, 6)
Number of duplicated rows :  2654
Shape after :  (127212, 6)


### Dropping sources and companies under threshold

In [ ]:
thresh_source = 30
thresh_company = 20

In [ ]:
# Dropping sources that wrote less than thresh_source articles
df = df.groupby('source').filter(lambda x: len(x) > thresh_source).reset_index(drop=True)
print('Number of sources remaining =>',len(df['source'].unique()))
print(df.shape)

Number of sources remaining => 75
(127090, 6)


In [ ]:
# Dropping companies that have less than thresh_company artciles about them
df = df.groupby('company').filter(lambda x: len(x) > thresh_company).reset_index(drop=True)
print('Number of companies remaining =>',len(df['company'].unique()))
print(df.shape)

Number of companies remaining => 75
(127048, 6)


### Basic pre-processing steps

In [ ]:
# Functions that removes stopwords from text
stoplist= set(stopwords.words("english"))

def remove_stopwords(text):
    tokens= word_tokenize(text)
    res = " ".join(w.lower() for w in tokens if not w.lower() in stoplist)
    return res

In [ ]:
# Removes all special characters and numericals leaving the alphabets
def clean(text):
    text = re.sub('[^A-Za-z]+', ' ', text)
    return text

In [ ]:
# Removes all punctuation that is irrelevant for textual analysis
def remove_punctuation(text):
    final = "".join(u for u in text if u not in ("?", ".", ";", ":",  "!",'"'))
    return final

In [ ]:
# Function that lemmatizes the text
lemmatizer = WordNetLemmatizer()

def lemmatize_sentence(text):
  word_list = nltk.word_tokenize(text)
  lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
  return lemmatized_output

In [ ]:
# Function that formats string to become more readable
def format_strings(text):
  text = re.sub('[^A-Za-z0-9]', ' ', text)
  return text

In [ ]:
# Function that applies all previously mentioned steps
def pre_processing(text):
    text = remove_punctuation(text)
    text = clean(text)
    text = remove_stopwords(text)
    text = lemmatize_sentence(text)
    return text

In [ ]:
# We apply the pre-processing on the articles and we format the source and title
df['text'] = df['text'].apply(pre_processing)
df['source'] = df['source'].apply(format_strings)
df['title'] = df['title'].apply(format_strings)

# Save output to xlsx

In [ ]:
# We save pre-processed output to xlsx to be able to re-use it
df.to_excel('Pre Processing/clean_dataset.xlsx', index=False)